In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Define the path to your dataset folder
data_path = "/content/drive/MyDrive/translation_data/"

# Load datasets directly from Google Drive
import pandas as pd

train_df = pd.read_csv(data_path + "train_data.csv")
val_df = pd.read_csv(data_path + "val_data.csv")
test_df = pd.read_csv(data_path + "test_data.csv")


In [11]:
!pip uninstall -y wandb



Found existing installation: wandb 0.19.9
Uninstalling wandb-0.19.9:
  Successfully uninstalled wandb-0.19.9


In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/translation_data/train_data.csv')
print(df.head())


                English             Italian
0         MEMBRANE UD-1       MEMBRANA UD-1
1      PLASTIC BASE LT8   BASE CABINATO LT8
2      COMMUTATOR 6 POS   COMMUTATORE 6 POS
3     MOTOR CASING K120  CARTER MOTORE K120
4  SCREW F3,5X13 N11725                VITE


In [4]:
!pip install transformers datasets sentencepiece sacrebleu evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict


# Load the datasets
train_df = pd.read_csv('/content/drive/MyDrive/translation_data/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/translation_data/val_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/translation_data/test_data.csv')

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Reduce dataset
train_dataset = train_dataset.select(range(min(300, len(train_dataset))))
val_dataset = val_dataset.select(range(min(50, len(val_dataset))))
test_dataset = test_dataset.select(range(min(50, len(test_dataset))))

# Verify the new sizes
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


from transformers import MBart50TokenizerFast

model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="it_IT")

max_length = 128

def tokenize_data(example):
    inputs = tokenizer(example['English'], max_length=max_length, truncation=True, padding="max_length")
    targets = tokenizer(example['Italian'], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_val = val_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_test = test_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])




Train size: 300
Validation size: 50
Test size: 50


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

# Setup Data Collator and Metrics

In [6]:
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_checkpoint, padding=True)

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    return {"bleu": result["score"]}


#Initialize the mBART Model

In [7]:
from transformers import MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

#Define Training Arguments

In [8]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
       output_dir="mbart-eng-it-domain-specific",
       eval_strategy="epoch",
       save_strategy="epoch",
       learning_rate=2e-5,
       per_device_train_batch_size=4,  #  batch size
       per_device_eval_batch_size=4,  #  batch size
       num_train_epochs=3,
       weight_decay=0.01,
       predict_with_generate=True,
       save_total_limit=2,
       fp16=True,
       report_to=None
   )

# Initialize the Trainer

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-9-cd3df47410e1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


#Start training

In [ ]:
trainer.train()


# Disable wandb to train free

#Evaluate the model

In [ ]:
eval_results = trainer.evaluate(tokenized_test)
print(f"BLEU score on test set: {eval_results['eval_bleu']:.2f}")


NameError: name 'trainer' is not defined

# Save model and tokenizer

In [ ]:
trainer.save_model("mbart-eng-it-trained")
tokenizer.save_pretrained("mbart-eng-it-trained")


#Inference - Testing translations

In [ ]:
# from transformers import pipeline

# translator = pipeline(
#     "translation",
#     model="mbart-eng-it-trained",
#     tokenizer="mbart-eng-it-trained",
#     src_lang="en_XX",
#     tgt_lang="it_IT"
# )

# sentence = "This is a domain-specific sentence for translation."
# translated_sentence = translator(sentence)[0]['translation_text']
# print(f"Translated sentence: {translated_sentence}")
